<a href="https://colab.research.google.com/github/AditPradana36/GSV_panorama_downloader/blob/main/GSV_panorama_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌐 **Google Street View Image Downloader**

This Google Colab notebook allows you to download Street View images from Google Maps using latitude and longitude coordinates provided in a CSV file. It supports both standard and panoramic downloads, saves images to a specified folder, and generates a metadata CSV.

# ✨ Features

✉️ Interactive parameters using @param form inputs

📷 Standard or 360° panoramic Street View image download

✔️ Automatic naming of images based on id column

📅 Metadata logging (date, pano ID, download time, etc.)

📂 Customizable image size, pitch, field of view (FOV), and radius



# 📄 CSV Format Required

Your CSV must include the following columns:


```
id,lat,lon
1,-6.200000,106.816666
2,-6.174465,106.822745
```

⚠️ id values must be numeric or convertible to integers (e.g., 1.0 becomes 1).

# **▶️ How to Use the Notebook**

# **📚 1. Install Dependencies**

This block installs the required libraries requests and pillow, and imports essential modules for HTTP requests, image handling, file writing, and CSV/dataframe processing.

In [ ]:
# ============================================
# 🛠 Install dependencies
# ============================================
!pip install requests pillow

import os
import csv
import datetime
import requests
from io import BytesIO
from PIL import Image
from google.colab import files
import pandas as pd

# **📂 2. Set Parameters**

Use this form to set your Google Maps API key, the CSV file path, image settings (like size, pitch, FOV), and output file names.
These parameters are editable in Google Colab through a form interface.

In [ ]:
# ============================================
# 📌 Parameters - Edit these using Colab form
# ============================================

api_key = "YOUR_GOOGLE_API"          # @param {type:"string"}
csv_path = "copy your csv path"      # @param {type:"string"}
output_folder = "copy your image output path"  # @param {type:"string"}
panorama = True                               # @param {type:"boolean"}
image_size = "640x640"                        # @param {type:"string"}
fov = 90                                      # @param {type:"integer"}
pitch = 0                                     # @param {type:"integer"}
radius = 30                                   # @param {type:"integer"}
source = "default"                            # @param {type:"string"}
metadata_output = "copy your metadata output path"     # @param {type:"string"}


# **🔄 3. Process the CSV and Start Download**

In [ ]:
# ============================================
# 📦 Street View Downloader Class
# ============================================

class StreetViewDownloader:
    BASE_IMAGE_URL = "https://maps.googleapis.com/maps/api/streetview"
    BASE_METADATA_URL = "https://maps.googleapis.com/maps/api/streetview/metadata"

    def __init__(self, api_key):
        self.api_key = api_key

    def _get_image_response(self, lat, lng, params):
        url = f"{self.BASE_IMAGE_URL}?location={lat},{lng}&key={self.api_key}"
        for key, value in params.items():
            url += f"&{key}={value}"
        return requests.get(url)

    def get_image_metadata(self, lat, lng, radius=50, source='default'):
        params = {
            'location': f"{lat},{lng}",
            'radius': radius,
            'source': source,
            'key': self.api_key
        }
        response = requests.get(self.BASE_METADATA_URL, params=params)
        return response.json() if response.status_code == 200 else {"status": "ERROR", "error": response.text}

    def download_image(self, lat, lng, output_folder=".", image_id="image", panorama=False, params=None):
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        if panorama:
            imgs = []
            for heading in [0, 90, 180, 270]:
                params['heading'] = heading
                response = self._get_image_response(lat, lng, params)
                if response.status_code != 200:
                    print(f"Failed at heading {heading} | {response.text}")
                    return False
                imgs.append(Image.open(BytesIO(response.content)))

            total_width = sum(img.width for img in imgs)
            max_height = max(img.height for img in imgs)
            panorama_img = Image.new("RGB", (total_width, max_height))
            x_offset = 0
            for img in imgs:
                panorama_img.paste(img, (x_offset, 0))
                x_offset += img.width

            filename = f"id_{image_id}_panorama.jpg"
            panorama_img.save(os.path.join(output_folder, filename))
        else:
            params['heading'] = 0
            response = self._get_image_response(lat, lng, params)
            if response.status_code != 200:
                print(f"Failed to download {lat},{lng} | {response.text}")
                return False
            filename = f"id_{image_id}.jpg"
            with open(os.path.join(output_folder, filename), 'wb') as f:
                f.write(response.content)

        return filename

# **💾 4. Output Summary**

In [ ]:
# ============================================
# 🔁 Process All Rows in CSV
# ============================================

downloader = StreetViewDownloader(api_key)

os.makedirs(output_folder, exist_ok=True)

with open(metadata_output, 'w', newline='') as meta_file:
    fieldnames = ['image_id', 'filename', 'lat', 'lon', 'panorama', 'capture_date', 'pano_id', 'status', 'download_timestamp']
    writer = csv.DictWriter(meta_file, fieldnames=fieldnames)
    writer.writeheader()

    df = pd.read_csv(csv_path)
    for idx, row in df.iterrows():
        lat, lng = row['lat'], row['lon']
        image_id = str(int(float(row['id'])))


        metadata = downloader.get_image_metadata(lat, lng, radius=radius, source=source)
        if metadata.get('status') != "OK":
            print(f"Skipping ID {image_id} ({lat},{lng}) due to metadata status: {metadata.get('status')}")
            continue

        params = {
            'size': image_size,
            'fov': fov,
            'pitch': pitch,
            'radius': radius,
            'source': source
        }

        filename = downloader.download_image(lat, lng, output_folder=output_folder, image_id=image_id, panorama=panorama, params=params)
        if not filename:
            continue

        writer.writerow({
            'image_id': image_id,
            'filename': filename,
            'lat': lat,
            'lon': lng,
            'panorama': panorama,
            'capture_date': metadata.get('date', ''),
            'pano_id': metadata.get('pano_id', ''),
            'status': metadata.get('status'),
            'download_timestamp': datetime.datetime.now().isoformat()
        })

print(f"✅ All images downloaded to `{output_folder}`")
print(f"📄 Metadata saved to `{metadata_output}`")
files.download(metadata_output)

✅ All images downloaded to `/content/downloaded_images`
📄 Metadata saved to `/content/metadata.csv`


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>